In [1]:
import numpy as np
import dgl
import os

import torch
from dgl.data.utils import download, extract_archive, get_download_dir, _get_dgl_url

from torch.utils.data import Dataset

import warnings
import torch.utils.data as data
from PIL import Image
import os
import os.path
import gzip
import numpy as np
import torch
import codecs
from torchvision import transforms

import networkx as nx


class GraphTransform:
    def __init__(self, device):
        self.adj = nx.to_numpy_matrix(nx.grid_2d_graph(28, 28))

    def __call__(self, img):
        return self.adj, \
               torch.tensor(img).reshape(-1, 1)

Using backend: pytorch


In [2]:
import atexit
import shutil

from tensorboardX import SummaryWriter

writer: SummaryWriter = None
e: int = None
train_iter : int = 0
test_iter : int = 0

@atexit.register
def del_tensorboard():
    option = input("Delete?")
    if option.lower() == 'd':
        shutil.rmtree(writer.log_dir)
        print("DELETED!")


In [43]:
import numpy as np
import dgl
import os

import torch
from dgl.data.utils import download, extract_archive, get_download_dir, _get_dgl_url

from torch.utils.data import Dataset


class TUDataset(Dataset):
    _url = r"https://ls11-www.cs.uni-dortmund.de/people/morris/graphkerneldatasets/{}.zip"

    def __init__(self, name):

        self.name = name

        self.extract_dir = self._download()
        DS_edge_list = torch.from_numpy(self._idx_from_zero(
            np.loadtxt(self._file_path("A"), delimiter=",", dtype=int)))
        DS_indicator = torch.from_numpy(self._idx_from_zero(
            np.loadtxt(self._file_path("graph_indicator"), dtype=int)))
        DS_graph_labels = torch.from_numpy(self._idx_from_zero(
            np.loadtxt(self._file_path("graph_labels"), dtype=int)))

        g = dgl.DGLGraph()
        g.add_nodes(int(DS_edge_list.max()) + 1)
        g.add_edges(DS_edge_list[:, 0], DS_edge_list[:, 1])
        #print('DS_edge_list[:, 0], DS_edge_list[:, 1]  ',DS_edge_list[:, 0], DS_edge_list[:, 1])
        #tensor([    1,     2,     3,  ..., 19556, 19561, 19562]) tensor([    0,     0,     0,  ..., 19579, 19579, 19579])
        node_idx_list = []
        for idx in range(torch.max(DS_indicator) + 1):
            node_idx = torch.where(DS_indicator == idx)
            node_idx_list.append(node_idx[0])
        self.graph_lists =[ g.subgraph(i) for i in node_idx_list]
        self.graph_labels = DS_graph_labels
        print('DS_graph_labels:',len(DS_graph_labels),'DS_edge_list[:, 0]:',len(DS_edge_list[:, 0]))
        try:
            DS_node_labels = self.to_onehot(
                torch.from_numpy(self._idx_from_zero(np.loadtxt(self._file_path("node_labels"), dtype=int))))
            for idxs, g in zip(node_idx_list, self.graph_lists):
                g.ndata['feat'] = DS_node_labels[idxs, :]
        except IOError:
            print("No Node Label Data")

        try:
            DS_node_attr = torch.from_numpy(np.loadtxt(self._file_path("node_attributes"), delimiter=","))
            cnt=0
            for idxs, g in zip(node_idx_list, self.graph_lists):
                g.ndata['feat'] = DS_node_attr[idxs, :]
                print(' DS_node_attr[idxs, :]',len( DS_node_attr[idxs, :]), DS_node_attr[idxs, :].shape)
                print('cnt ',cnt)
                cnt+=1
        except IOError:
            print("No Node Attribute Data")

    def __getitem__(self, idx):
        g = self.graph_lists[idx]
        return (g.adjacency_matrix().to_dense(), g.ndata['feat']), self.graph_labels[idx]

    def __len__(self):
        return len(self.graph_lists)

    def _download(self):
        download_dir = get_download_dir()
        zip_file_path = r'/root/server/Semantic-Loss/complex_constraints/curriculum learning/graphkerneldatasets/ENZYMES.zip'
        extract_dir = r'/root/server/Semantic-Loss/complex_constraints/curriculum learning/graphkerneldatasets/ENZYMES'
        extract_archive(zip_file_path, extract_dir)
        return extract_dir

    def _file_path(self, category):
        return os.path.join(self.extract_dir, self.name, "{}_{}.txt".format(self.name, category))

    @staticmethod
    def _idx_from_zero(idx_tensor):
        return idx_tensor - np.min(idx_tensor)

    @staticmethod
    def to_onehot(label_tensor):
        label_num = label_tensor.shape[0]
        assert torch.min(label_tensor) == 0
        one_hot_tensor = torch.zeros((label_num, torch.max(label_tensor) + 1))
        one_hot_tensor[torch.arange(label_num), label_tensor] = 1
        return one_hot_tensor


class CollateFn:
    def __init__(self, device='cpu'):
        self.device = device

    def __call__(self, batch):
        adj_tensor_list = []
        features_list = []
        mask_list = []
        # (adj, features), labels = list(zip(*batch))
        max_num_nodes = max([g[0][0].shape[0] for g in batch])
        labels = []
        for (A, F), L in batch:
            labels.append(L)
            length = A.shape[0]
            pad_len = max_num_nodes - length
            adj_tensor_list.append( torch.from_numpy(np.pad(A, ((0, pad_len), (0, pad_len)), mode='constant')))
            features_list.append( torch.from_numpy(np.pad(F, ((0, pad_len), (0, 0)), mode='constant')))
            mask = torch.zeros(max_num_nodes)
            mask[:length] = 1
            mask_list.append(mask)
        return torch.stack(adj_tensor_list, 0).float().to(self.device), \
               torch.stack(features_list, 0).float().to(self.device), \
               torch.stack(mask_list, 0).float().to(self.device), \
               torch.stack(labels, 0).long().to(self.device)


In [44]:
import seaborn
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import scipy
import matplotlib.pyplot as plt


class BatchedGraphSAGE(nn.Module):
    def __init__(self, infeat, outfeat, device='cpu', use_bn=True, mean=False, add_self=False):
        super().__init__()
        self.add_self = add_self
        self.use_bn = use_bn
        self.device = device
        self.mean = mean
        self.W = nn.Linear(infeat, outfeat, bias=True)
        nn.init.xavier_uniform_(self.W.weight, gain=nn.init.calculate_gain('relu'))

    def forward(self, x, adj, mask=None):
        if self.add_self:
            adj = adj + torch.eye(adj.size(0)).to(self.device)

        if self.mean:
            adj = adj / adj.sum(1, keepdim=True)

        h_k_N = torch.matmul(adj, x)
        h_k = self.W(h_k_N)
        h_k = F.normalize(h_k, dim=2, p=2)
        h_k = F.relu(h_k)
        if self.use_bn:
            self.bn = nn.BatchNorm1d(h_k.size(1)).to(self.device)
            h_k = self.bn(h_k)
        if mask is not None:
            h_k = h_k * mask.unsqueeze(2).expand_as(h_k)
        return h_k


class BatchedDiffPool(nn.Module):
    def __init__(self, nfeat, nnext, nhid, is_final=False, device='cpu', link_pred=False):
        super(BatchedDiffPool, self).__init__()
        self.link_pred = link_pred
        self.device = device
        self.is_final = is_final
        self.embed = BatchedGraphSAGE(nfeat, nhid, device=self.device, use_bn=True)
        self.assign_mat = BatchedGraphSAGE(nfeat, nnext, device=self.device, use_bn=True)
        self.log = {}
        self.link_pred_loss = 0
        self.entropy_loss = 0

    def forward(self, x, adj, mask=None, log=False):
        z_l = self.embed(x, adj)
        s_l = F.softmax(self.assign_mat(x, adj), dim=-1)
        if log:
            self.log['s'] = s_l.cpu().numpy()
        xnext = torch.matmul(s_l.transpose(-1, -2), z_l)
        anext = (s_l.transpose(-1, -2)).matmul(adj).matmul(s_l)
        if self.link_pred:
            # TODO: Masking padded s_l
            self.link_pred_loss = (adj - s_l.matmul(s_l.transpose(-1, -2))).norm(dim=(1, 2))
            self.entropy_loss = torch.distributions.Categorical(probs=s_l).entropy()
            if mask is not None:
                self.entropy_loss = self.entropy_loss * mask.expand_as(self.entropy_loss)
            self.entropy_loss = self.entropy_loss.sum(-1)
        return xnext, anext


class Classifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.classifier = nn.Sequential(nn.Linear(30, 50),
                                        nn.ReLU(),
                                        nn.Linear(50, n_classes))

    def forward(self, x):
        return self.classifier(x)


class BatchedModel(nn.Module):
    def __init__(self, pool_size, device, input_shape, n_classes, link_pred=False):
        super().__init__()
        self.input_shape = input_shape
        self.link_pred = link_pred
        self.device = device
        self.layers = nn.ModuleList([
            BatchedGraphSAGE(input_shape, 30, device=self.device),
            BatchedGraphSAGE(30, 30, device=self.device),
            BatchedDiffPool(30, pool_size, 30, device=self.device, link_pred=link_pred),
            BatchedGraphSAGE(30, 30, device=self.device),
            BatchedGraphSAGE(30, 30, device=self.device),
            # BatchedDiffPool(30, 1, 30, is_final=True, device=self.device)
        ])
        self.classifier = Classifier(n_classes)
        # writer.add_text(str(vars(self)))

    def forward(self, x, adj, mask):
        for layer in self.layers:
            if isinstance(layer, BatchedGraphSAGE):
                if mask.shape[1] == x.shape[1]:
                    x = layer(x, adj, mask)
                else:
                    x = layer(x, adj)
            elif isinstance(layer, BatchedDiffPool):
                # TODO: Fix if condition
                if mask.shape[1] == x.shape[1]:
                    x, adj = layer(x, adj, mask)
                else:
                    x, adj = layer(x, adj)

        # x = x * mask
        readout_x = x.sum(dim=1)
        return readout_x

    def loss(self, output, labels):
        criterion = nn.CrossEntropyLoss()
        loss = criterion(output, labels)
        if self.link_pred:
            for layer in self.layers:
                if isinstance(layer, BatchedDiffPool):
                    loss = loss + layer.link_pred_loss.mean() + layer.entropy_loss.mean()

        return loss


In [45]:
import argparse

import torch
import torch.optim as optim
from torchvision.datasets import MNIST
from tqdm import tqdm
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader, random_split


In [46]:


class para:
    def __init__(self):
        self.no_cuda=False
        self.epochs =700
        self.link_pred=False
        self.dataset='ENZYMES'
        self.batch_size=256
        self.train_ratio=0.9
        self.pool_ratio=0.25
args = para()
device = "cuda" if not args.no_cuda and torch.cuda.is_available() else "cpu"

dataset = TUDataset(args.dataset)
# dataset = MNIST(root="~/.torch/data/", transform=GraphTransform(device), download=True)
dataset_size = len(dataset)
train_size = int(dataset_size * args.train_ratio)
test_size = dataset_size - train_size
max_num_nodes = max([item[0][0].shape[0] for item in dataset])
n_classes = int(max([item[1] for item in dataset])) + 1
train_data, test_data = random_split(dataset, (train_size, test_size))
input_shape = int(dataset[0][0][1].shape[-1])
train_loader = DataLoader(train_data, batch_size=args.batch_size, shuffle=True,
                          collate_fn=CollateFn(device))
test_loader = DataLoader(test_data, batch_size=args.batch_size, shuffle=True,
                         collate_fn=CollateFn(device))

model = BatchedModel(pool_size=int(max_num_nodes * args.pool_ratio), device=device,
                     link_pred=args.link_pred, input_shape=input_shape, n_classes=n_classes).to(
    device)
model.train()
optimizer = optim.Adam(model.parameters())

for e in tqdm(range(args.epochs)):
    e = e
    epoch_losses_list = []
    true_sample = 0
    model.train()
    for i, (adj, features, masks, batch_labels) in enumerate(train_loader):
        train_iter += 1
        graph_feat = model(features, adj, masks)
        output = model.classifier(graph_feat)
        loss = model.loss(output, batch_labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 2.0)
        optimizer.step()
        optimizer.zero_grad()

        epoch_losses_list.append(loss.item())
        iter_true_sample = (output.argmax(dim=1).long() == batch_labels.long()). \
            float().sum().item()
        iter_acc = float(iter_true_sample) / output.shape[0]
        print(f"{train_iter} iter train acc: {iter_acc}")
        true_sample += iter_true_sample

    acc = true_sample / train_size
    tqdm.write(f"Epoch:{e}  \t train_acc:{acc:.2f}")

    test_loss_list = []
    true_sample = 0
    model.eval()
    with torch.no_grad():
        for i, (adj, features, masks, batch_labels) in enumerate(test_loader):
            test_iter += 1
            graph_feat = model(features, adj, masks)
            output = model.classifier(graph_feat)
            loss = model.loss(output, batch_labels)
            test_loss_list.append(loss.item())
            iter_true_sample = (output.argmax(dim=1).long() == batch_labels.long()). \
                float().sum().item()
            iter_acc = float(iter_true_sample) / output.shape[0]
            print(f"{test_iter} iter test acc: {iter_acc}")
            true_sample += iter_true_sample
    acc = true_sample / test_size
    tqdm.write(f"Epoch:{e}  \t val_acc:{acc:.2f}")

    # Visualize
    # adj, features, masks, batch_labels = test_loalder
    # model()




DS_graph_labels: 600 DS_edge_list[:, 0]: 74564
 DS_node_attr[idxs, :] 37 torch.Size([37, 18])
cnt  0
 DS_node_attr[idxs, :] 23 torch.Size([23, 18])
cnt  1
 DS_node_attr[idxs, :] 25 torch.Size([25, 18])
cnt  2
 DS_node_attr[idxs, :] 24 torch.Size([24, 18])
cnt  3
 DS_node_attr[idxs, :] 23 torch.Size([23, 18])
cnt  4
 DS_node_attr[idxs, :] 24 torch.Size([24, 18])
cnt  5
 DS_node_attr[idxs, :] 26 torch.Size([26, 18])
cnt  6
 DS_node_attr[idxs, :] 88 torch.Size([88, 18])
cnt  7
 DS_node_attr[idxs, :] 23 torch.Size([23, 18])
cnt  8
 DS_node_attr[idxs, :] 32 torch.Size([32, 18])
cnt  9
 DS_node_attr[idxs, :] 4 torch.Size([4, 18])
cnt  10
 DS_node_attr[idxs, :] 14 torch.Size([14, 18])
cnt  11
 DS_node_attr[idxs, :] 42 torch.Size([42, 18])
cnt  12
 DS_node_attr[idxs, :] 41 torch.Size([41, 18])
cnt  13
 DS_node_attr[idxs, :] 36 torch.Size([36, 18])
cnt  14
 DS_node_attr[idxs, :] 55 torch.Size([55, 18])
cnt  15
 DS_node_attr[idxs, :] 40 torch.Size([40, 18])
cnt  16
 DS_node_attr[idxs, :] 38 torc

 DS_node_attr[idxs, :] 52 torch.Size([52, 18])
cnt  282
 DS_node_attr[idxs, :] 48 torch.Size([48, 18])
cnt  283
 DS_node_attr[idxs, :] 50 torch.Size([50, 18])
cnt  284
 DS_node_attr[idxs, :] 42 torch.Size([42, 18])
cnt  285
 DS_node_attr[idxs, :] 43 torch.Size([43, 18])
cnt  286
 DS_node_attr[idxs, :] 41 torch.Size([41, 18])
cnt  287
 DS_node_attr[idxs, :] 39 torch.Size([39, 18])
cnt  288
 DS_node_attr[idxs, :] 38 torch.Size([38, 18])
cnt  289
 DS_node_attr[idxs, :] 62 torch.Size([62, 18])
cnt  290
 DS_node_attr[idxs, :] 60 torch.Size([60, 18])
cnt  291
 DS_node_attr[idxs, :] 96 torch.Size([96, 18])
cnt  292
 DS_node_attr[idxs, :] 54 torch.Size([54, 18])
cnt  293
 DS_node_attr[idxs, :] 124 torch.Size([124, 18])
cnt  294
 DS_node_attr[idxs, :] 126 torch.Size([126, 18])
cnt  295
 DS_node_attr[idxs, :] 122 torch.Size([122, 18])
cnt  296
 DS_node_attr[idxs, :] 24 torch.Size([24, 18])
cnt  297
 DS_node_attr[idxs, :] 41 torch.Size([41, 18])
cnt  298
 DS_node_attr[idxs, :] 49 torch.Size([49, 

 DS_node_attr[idxs, :] 40 torch.Size([40, 18])
cnt  563
 DS_node_attr[idxs, :] 19 torch.Size([19, 18])
cnt  564
 DS_node_attr[idxs, :] 14 torch.Size([14, 18])
cnt  565
 DS_node_attr[idxs, :] 13 torch.Size([13, 18])
cnt  566
 DS_node_attr[idxs, :] 13 torch.Size([13, 18])
cnt  567
 DS_node_attr[idxs, :] 44 torch.Size([44, 18])
cnt  568
 DS_node_attr[idxs, :] 45 torch.Size([45, 18])
cnt  569
 DS_node_attr[idxs, :] 46 torch.Size([46, 18])
cnt  570
 DS_node_attr[idxs, :] 47 torch.Size([47, 18])
cnt  571
 DS_node_attr[idxs, :] 47 torch.Size([47, 18])
cnt  572
 DS_node_attr[idxs, :] 42 torch.Size([42, 18])
cnt  573
 DS_node_attr[idxs, :] 51 torch.Size([51, 18])
cnt  574
 DS_node_attr[idxs, :] 27 torch.Size([27, 18])
cnt  575
 DS_node_attr[idxs, :] 27 torch.Size([27, 18])
cnt  576
 DS_node_attr[idxs, :] 60 torch.Size([60, 18])
cnt  577
 DS_node_attr[idxs, :] 28 torch.Size([28, 18])
cnt  578
 DS_node_attr[idxs, :] 38 torch.Size([38, 18])
cnt  579
 DS_node_attr[idxs, :] 23 torch.Size([23, 18])
c

  0%|          | 0/700 [00:00<?, ?it/s]

2295 iter train acc: 0.1953125


...

  1%|▏         | 9/700 [00:05<06:37,  1.74it/s]

2320 iter train acc: 0.23046875
2321 iter train acc: 0.2857142857142857
Epoch:8  	 train_acc:0.24
771 iter test acc: 0.21666666666666667
Epoch:8  	 val_acc:0.22
2322 iter train acc: 0.25


  1%|▏         | 9/700 [00:05<07:15,  1.59it/s]

2323 iter train acc: 0.23828125
2324 iter train acc: 0.32142857142857145
Epoch:9  	 train_acc:0.25


KeyboardInterrupt: 

In [1]:

import configparser
import numpy as np
import random
import gym
import sys
sys.path.insert(0,'/root/multiagent_gnn_policies/gym-flock/')
import gym_flock
import torch
import sys

AirSim not installed.


In [2]:
gym_flock.envs

AttributeError: module 'gym_flock' has no attribute 'envs'

In [4]:
!cat /root/multiagent_gnn_policies/gym-flock/gym_flock/__init__.py

from gym.envs.registration import register

register(
    id='ExploreEnv-v0',
    entry_point='gym_flock.envs.spatial:ExploreEnv',
    max_episode_steps=100000,
)

register(
    id='ExploreFullEnv-v0',
    entry_point='gym_flock.envs.spatial:ExploreFullEnv',
    max_episode_steps=100000,
)

register(
    id='ExploreEnv-v1',
    entry_point='gym_flock.envs.spatial:ExploreEnv',
    max_episode_steps=100000,
)

register(
    id='CoverageFull-v0',
    entry_point='gym_flock.envs.spatial:CoverageFullEnv',
    max_episode_steps=100000,
)

register(
    id='CoverageARL-v1',
    entry_point='gym_flock.envs.spatial:CoverageARLEnv',
    max_episode_steps=100000,
)


register(
    id='CoverageARL-v0',
    entry_point='gym_flock.envs.spatial:CoverageARLEnv',
    max_episode_steps=100000,
)

register(
    id='Coverage-v0',
    entry_point='gym_flock.envs.spatial:CoverageEnv',
    max_episode_steps=75,
)


register(
    id='Shepherding-v0',
    entry_point='gym_flock.envs.shepherding:ShepherdingEnv'

In [6]:
import sys
sys.path.insert(0,'/root/marl_transfer/mape/multiagent/')
from multi_discrete import MultiDiscrete

In [5]:
dir(gym.spaces)

['Box',
 'Dict',
 'Discrete',
 'MultiBinary',
 'MultiDiscrete',
 'Space',
 'Tuple',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'box',
 'dict',
 'discrete',
 'flatdim',
 'flatten',
 'multi_binary',
 'multi_discrete',
 'space',
 'tuple',
 'unflatten',
 'utils']